In [1]:
# 실습 2: 멀티미디어 데이터 수집하기 


In [2]:
! pip install pillow

In [ ]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import re
import os
import pandas as pd
from urllib.parse import urlparse, parse_qs, unquote

headers = {'User-Agent': 'Mozilla/5.0'}

#url: https://www.moviechart.co.kr/rank/realtime/index/image
movie_ranking = requests.get('https://www.moviechart.co.kr/rank/realtime/index/image')

# 이미지 저장할 폴더 생성 
image_dir = 'images'
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# 파일명애서 사용할 수 없는 문자빼기 
pattern = r'[\\/:"*?<>|]'

if movie_ranking.status_code == 200:
    print('영화 정보를 출력합니다.')
    soup = BeautifulSoup(movie_ranking.content, 'html.parser')
    title_list = soup.select('div.movie-title > h3 > a')
    image_list = soup.select(' div.movieBox a > img') # > 잘 사용해서 쓰기 _ 직계
    print(f"수집한 영화 수: {len(title_list)}")
    print(f"수집한 영화 이미지 수: {len(image_list)}")

    # 결과 저장용 리스트
    movie_data = []
    

    for title, image in zip(title_list, image_list):
        title_text = title.text.strip()
        raw_src = image.get('src')

        # 'source=' 뒤에 있는 원본 이미지 링크만 추출
        if 'source=' in raw_src:
            parsed = urlparse(raw_src)
            real_image_url = parse_qs(parsed.query).get('source', [''])[0]
            image_url = unquote(real_image_url)
        else:
            image_url = raw_src if raw_src.startswith('http') else 'https://www.moviechart.co.kr' + raw_src

        movie_data.append({
            'title': title_text,
            'image_url': image_url  # 원본 이미지 주소만 CSV에 기록
        })

    # CSV 저장 (이미지 저장은 안 함)
    df = pd.DataFrame(movie_data)
    df.to_csv("movie_images.csv", encoding="utf-8-sig", index=False)
    print(" movie_images.csv 저장 완료!")

else:
    print(" 페이지 요청 실패")

영화 정보를 출력합니다.
수집한 영화 수: 20
수집한 영화 이미지 수: 20
 movie_images.csv 저장 완료!
